In [1]:
import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments

df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                

## How many different cases are there, by department?

In [24]:
(
df.groupBy('department')
    .count()
    .show()
)
# those are all the cases by dept, trying to get distinct cases

+--------------------+------+
|          department| count|
+--------------------+------+
|         Solid Waste|279270|
|Animal Care Services|116915|
|Trans & Cap Impro...| 96193|
|  Parks & Recreation| 19907|
|    Customer Service|  2849|
|        Metro Health|  5163|
|        City Council|    33|
|DSD/Code Enforcement|321374|
+--------------------+------+



In [22]:
(
df.groupBy('department')
    .agg(sum("service_request_type"))
    .show()
)

+--------------------+-------------------------+
|          department|sum(service_request_type)|
+--------------------+-------------------------+
|         Solid Waste|                     null|
|Animal Care Services|                     null|
|Trans & Cap Impro...|                     null|
|  Parks & Recreation|                     null|
|    Customer Service|                     null|
|        Metro Health|                     null|
|        City Council|                     null|
|DSD/Code Enforcement|                     null|
+--------------------+-------------------------+



## Does the percentage of cases that are late vary by department?
## On average, how late are the late cases by department?
## What is the service type that is the most late? Just for Parks & Rec?
## For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.
## Does whether or not its a weekend matter for when a case is opened/closed?
## On average, how many cases are opened a day for the Customer Service department?
## Does the number of service requests for the solid waste department vary by day of the week?